In [2]:
import pandas as pd

In [3]:
df1 = pd.concat(pd.read_excel(r'R:\1. DATABASE\1. Bank\Mai Ngoc\Txns Report Test\Airpay_Transaction_Details_(2020-03-01-2020-03-08).1585730261.xlsx', sheet_name=None), ignore_index=True)

In [4]:
df2 = pd.concat(pd.read_excel(r'R:\1. DATABASE\1. Bank\Mai Ngoc\Txns Report Test\Airpay_Transaction_Details_(2020-03-08-2020-03-11).1585803083.xlsx', sheet_name=None), ignore_index=True)

In [5]:
df3 = pd.concat(pd.read_excel(r'R:\1. DATABASE\1. Bank\Mai Ngoc\Txns Report Test\Airpay_Transaction_Details_(2020-03-11-2020-03-15).1585803790.xlsx', sheet_name=None), ignore_index=True)

In [ ]:
df4 = pd.concat(pd.read_excel(r'R:\1. DATABASE\1. Bank\Mai Ngoc\Txns Report Test\Airpay_Transaction_Details_(2020-03-15-2020-03-19).1585804594.xlsx', sheet_name=None), ignore_index=True)

In [ ]:
df5 = pd.concat(pd.read_excel(r'R:\1. DATABASE\1. Bank\Mai Ngoc\Txns Report Test\Airpay_Transaction_Details_(2020-03-19-2020-03-23).1585805738.xlsx', sheet_name=None), ignore_index=True)

In [ ]:
df6 = pd.concat(pd.read_excel(r'R:\1. DATABASE\1. Bank\Mai Ngoc\Txns Report Test\Airpay_Transaction_Details_(2020-03-23-2020-03-25).1585797494.xlsx', sheet_name=None), ignore_index=True)

In [ ]:
df7 = pd.concat(pd.read_excel(r'R:\1. DATABASE\1. Bank\Mai Ngoc\Txns Report Test\Airpay_Transaction_Details_(2020-03-25-2020-03-27).1585796849.xlsx', sheet_name=None), ignore_index=True)

In [ ]:
df8 = pd.concat(pd.read_excel(r'R:\1. DATABASE\1. Bank\Mai Ngoc\Txns Report Test\Airpay_Transaction_Details_(2020-03-27-2020-04-01).1585741535.xlsx', sheet_name=None), ignore_index=True)

In [ ]:
df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

In [ ]:
import pyodbc 
import datetime as dt

In [ ]:
username='maingoc.dinh'
password='Maingoc01'
server = 'GA-9RC9BS2\SQLEXPRESS'
db='TVT_03_2020'
db_Giro='IS_Giro_03_2020'
db_GW='IS_IBanking_03_2020'

m=3
range_day=range(1,32,1) #chọn dài hơn 1 ngày
#range_day_GW=range(1,9,1) #chọn đúng kỳ với cuối tháng?
#TVT_day='23_31'

In [ ]:
# sql server 2017
conn_str = ('Driver={SQL Server};'
                'Server='+ server+';'
                'Database='+db+';'
                'UID='+username+';'
                'PWD='+password+';')

connection = pyodbc.connect(conn_str)
data=[]
for d in range_day:
    sql_statement = """SELECT * FROM [{2}].[dbo].[TVT_2020-{0:0>2d}-{1:0>2d}-2020-{0:0>2d}-{1:0>2d}]""".format(m,d,db)
    sql_data = pd.read_sql(sql_statement, connection)
    data.append(sql_data)

In [ ]:
TVT = pd.concat(data)

In [ ]:
#df.head()

In [ ]:
df['OrderID']=df['OrderID'].astype(str)

In [ ]:
#TVT.head()

In [ ]:
TVT_df = df.merge(TVT,left_on='OrderID',right_on='OrderID',how='left',indicator=True)

In [ ]:
TVT_add=TVT_df[TVT_df['_merge'].isin(['left_only'])&TVT['TopUp Type'].isin(['Bank Account','Account Balance','Ibanking','Offline Topup Partner'])]

In [ ]:
#TVT_add.to_excel(r'TVT_add.xlsx')

In [ ]:
#TVT_add_Bank_Acc = pd.read_excel(r'D:\TVT_add_Bank_Acc.xlsx')

In [ ]:
def Type(TopUp,PaymentInfo):
    if 'Cash Withdrawal' in PaymentInfo:
        i = 'withdraw'
    elif 'Shopee Wallet Topup' in TopUp:
        i='APA Shopee GW'
    elif 'Cash via iBanking' in PaymentInfo:
        i='GW'
    elif 'Cash via Direct Bank Channel Top-up' in PaymentInfo:
        i='API'
    elif 'Cash via Direct Debit' in PaymentInfo or 'Cash via Partner Top-up' in PaymentInfo:
        i='Payment'
    elif 'Cash Transfer' in PaymentInfo or 'Now Transfer' in PaymentInfo or 'Collection QR' in PaymentInfo:
        i='Transfer'
    else:
        i='Shopping'
    return(i)
        

In [ ]:
TVT_add['Type']=TVT_add.apply(lambda row:Type(row['TopUp_x'],row['Payment Infos']),axis=1)

In [ ]:
TVT_add.to_excel(r'TVT_add_Mar20.xlsx')